In [ ]:
import pandas as pd
from pathlib import Path

In [4]:

pd.set_option('display.float_format', lambda v: f'{v:,.2f}')

In [5]:
data_path = Path('datasets/usd_rates.csv')
df = pd.read_csv(data_path)
df['Date'] = pd.to_datetime(df['Date'], format='%d.%m.%Y', errors='coerce')
df = df.sort_values(['Ccy', 'Date']).dropna(subset=['Date']).reset_index(drop=True)
print(f'Satrlar: {len(df):,}; Ustunlar: {df.shape[1]}')
df.head()


Satrlar: 5,196; Ustunlar: 11


,id,Code,Ccy,CcyNm_RU,CcyNm_UZ,CcyNm_UZC,CcyNm_EN,Nominal,Rate,Diff,Date
0,1,840,USD,Доллар США,AQSH dollari,АҚШ доллари,US Dollar,1,"8,299.20",21.67,2018-11-27
1,1,840,USD,Доллар США,AQSH dollari,АҚШ доллари,US Dollar,1,"8,299.20",21.67,2018-11-27
2,1,840,USD,Доллар США,AQSH dollari,АҚШ доллари,US Dollar,1,"8,299.20",21.67,2018-11-27
3,1,840,USD,Доллар США,AQSH dollari,АҚШ доллари,US Dollar,1,"8,299.20",21.67,2018-11-27
4,1,840,USD,Доллар США,AQSH dollari,АҚШ доллари,US Dollar,1,"8,299.20",21.67,2018-11-27


In [6]:
gap_summary = []
for ccy, g in df.groupby('Ccy'):
    g = g.sort_values('Date')
    gaps = (g['Date'].diff().dt.days.fillna(0) - 1).clip(lower=0)
    gap_summary.append({'Ccy': ccy, 'missing_days': int(gaps.sum())})

gap_df = pd.DataFrame(gap_summary).sort_values('missing_days', ascending=False)
gap_df


,Ccy,missing_days
0,USD,1253


In [8]:
def fill_missing_days(group: pd.DataFrame) -> pd.DataFrame:
    group = group.sort_values('Date').drop_duplicates(subset='Date', keep='last')
    full_idx = pd.date_range(group['Date'].min(), group['Date'].max(), freq='D')
    filled = group.set_index('Date').reindex(full_idx).ffill()
    filled['Date'] = filled.index
    return filled.reset_index(drop=True)

filled_df = df.groupby('Ccy', group_keys=False).apply(fill_missing_days)
filled_df = filled_df.sort_values(['Ccy', 'Date']).reset_index(drop=True)

print(f'Oldingi satrlar: {len(df):,}; Yangi satrlar: {len(filled_df):,}')
filled_df.tail(10)


Oldingi satrlar: 5,196; Yangi satrlar: 2,570


C:\Users\zikru\AppData\Local\Temp\ipykernel_18348\1201277546.py:8: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  filled_df = df.groupby('Ccy', group_keys=False).apply(fill_missing_days)


,id,Code,Ccy,CcyNm_RU,CcyNm_UZ,CcyNm_UZC,CcyNm_EN,Nominal,Rate,Diff,Date
2560,1.00,840.00,USD,Доллар США,AQSH dollari,АҚШ доллари,US Dollar,1.00,"11,902.30",20.70,2025-11-30
2561,1.00,840.00,USD,Доллар США,AQSH dollari,АҚШ доллари,US Dollar,1.00,"11,940.95",38.65,2025-12-01
2562,1.00,840.00,USD,Доллар США,AQSH dollari,АҚШ доллари,US Dollar,1.00,"11,895.57",-45.38,2025-12-02
2563,1.00,840.00,USD,Доллар США,AQSH dollari,АҚШ доллари,US Dollar,1.00,"11,880.93",-14.64,2025-12-03
2564,1.00,840.00,USD,Доллар США,AQSH dollari,АҚШ доллари,US Dollar,1.00,"11,912.38",31.45,2025-12-04
2565,1.00,840.00,USD,Доллар США,AQSH dollari,АҚШ доллари,US Dollar,1.00,"11,963.23",50.85,2025-12-05
2566,1.00,840.00,USD,Доллар США,AQSH dollari,АҚШ доллари,US Dollar,1.00,"11,963.23",50.85,2025-12-06
2567,1.00,840.00,USD,Доллар США,AQSH dollari,АҚШ доллари,US Dollar,1.00,"11,963.23",50.85,2025-12-07
2568,1.00,840.00,USD,Доллар США,AQSH dollari,АҚШ доллари,US Dollar,1.00,"11,963.23",50.85,2025-12-08
2569,1.00,840.00,USD,Доллар США,AQSH dollari,АҚШ доллари,US Dollar,1.00,"11,990.09",26.86,2025-12-09


In [9]:
has_gap = (
    filled_df.sort_values(['Ccy', 'Date'])
    .groupby('Ccy')['Date']
    .diff()
    .dt.days
    .fillna(1)
    .gt(1)
    .any()
)
print('Bo`shliq qoldimi?:', bool(has_gap))
filled_df.tail()


Bo`shliq qoldimi?: False


,id,Code,Ccy,CcyNm_RU,CcyNm_UZ,CcyNm_UZC,CcyNm_EN,Nominal,Rate,Diff,Date
2565,1.00,840.00,USD,Доллар США,AQSH dollari,АҚШ доллари,US Dollar,1.00,"11,963.23",50.85,2025-12-05
2566,1.00,840.00,USD,Доллар США,AQSH dollari,АҚШ доллари,US Dollar,1.00,"11,963.23",50.85,2025-12-06
2567,1.00,840.00,USD,Доллар США,AQSH dollari,АҚШ доллари,US Dollar,1.00,"11,963.23",50.85,2025-12-07
2568,1.00,840.00,USD,Доллар США,AQSH dollari,АҚШ доллари,US Dollar,1.00,"11,963.23",50.85,2025-12-08
2569,1.00,840.00,USD,Доллар США,AQSH dollari,АҚШ доллари,US Dollar,1.00,"11,990.09",26.86,2025-12-09


In [10]:
output_path = Path('datasets/usd_rates_filled.csv')
filled_df.to_csv(output_path, index=False)
output_path


WindowsPath('datasets/usd_rates_filled.csv')